<hr>
<div style="background-color: lightgray; padding: 20px; color: black;">
<div>
<img src="https://th.bing.com/th/id/R.3cd1c8dc996c5616cf6e65e20b6bf586?rik=09aaLyk4hfbBiQ&riu=http%3a%2f%2fcidics.uanl.mx%2fwp-content%2fuploads%2f2016%2f09%2fcimat.png&ehk=%2b0brgMUkA2BND22ixwLZheQrrOoYLO3o5cMRqsBOrlY%3d&risl=&pid=ImgRaw&r=0" style="float: right; margin-right: 30px;" width="200"/> 
<font size="5.5" color="8C3061"><b>Design Pattern Singleton</b></font> <br>
<font size="5.5" color="8C3061"><b>Creational Design Patterns in Modern C++</b></font> 
</div>
<div style="text-align: left">  <br>
Edison David Serrano Cárdenas<br>
MSc en Matemáticas Aplicadas; CIMAT - Sede Guanajuato <br>
</div>

</div>
<hr>

# <font color="8C3061" >**Introduction**</font> 

This pattern is used when you want a class to have only one instance. 

Now what kind of classes will require only one instance? 

You may have a component that manages some underlying resource and that resource could be maybe a print spooler, maybe a database connection or even an application configuration.



<div class="alert alert-block alert-info">
<b>Note:</b> Now, if you create a class that manages the underlying resource, then the class should have only one instance because if there are multiple instances, then each instance will store its own state of the underlying resource. 

And when one instance modifies the resource, the other instance will not know about it. So this way the state of the underlying resource may get corrupted or it may fail to provide the service. So that's why this class should have only one instance. </div>

> Ensure a class only has one instance, and provide a global point of access to it.

So you might feel that this makes the Singleton instance behave like a global variable. And yes, you're right, it does behave like a global variable.

<font color="8C3061" >**Implementation**</font> 


- The class is made responsible for its own instance.
- It intercepts the call for construction and returns a single instance.
- Same instance is returned every time.
- Therefore, direct construction of object is disabled.
- The class creates its own instance which is provided to the clients.


<font color="8C3061" >**OMD Diagram:**</font> 


<center>
<img src="./images/uml_singleton.png" width="450"/> 
</center>

<font color="8C3061" >**Logger Implementation**</font> 

1. Add a class called Singleton.
2. The first thing we need to do is disable the construction and we can do that very easily by making the constructor private.
   - if your constructor body is empty, then use the default specifier on this constructor so that the Compiler automatically synthesizes the default implementation of the constructor. This implementation is going to be much faster than the user defined default constructor.   
3. Create the static instance of this class

***Example:***

We will create a logger clause that will be used to log the different states of the application as it executes.

If the application crashes, we can examine the logs and we can find out which part of the application caused the crash. 

`#pragma once` reemplace 
```c++
#ifndef _MYHEADER_H_
#define _MYHEADER_H_
...
#endif
```

The #pragma once directive has a very simple concept. The header file containing this directive is included only once even if the programmer includes it multiple times during a compilation. This directive works similar to the #include guard idiom. Use of #pragma once saves the program from multiple inclusion optimization.

***Logger Class:***

File: `Logger.h`
```c++
class Logger{
    FILE *m_pStream;
    std::string m_Tag;
public:
    Logger() ;
    ~Logger() ;
    void WriteLog(const char *pMessage) ;
    void SetTag(const char *pTag) ;
};
```

File: `main.cpp`

```c++
# include "Logger.h"
void OpenConection(){
    Logger lg;
    lg.WriteLog("Attempting to open a connection"); 
}

int main(){
    Logger lg;
    lg.SetTag("192.168.1.101");
    lg.WriteLog("Application has started");
    OpenConection();
    lg.WriteLog("Application is shutting down");
    return 0;
}
```

Output `applog.txt`
```txt
[] Attempting to open a connection
     [192.168.1.101] Application is shutting down

```

The problem is due to the fact that two instances are created and the constructor of each instance attempts to open the file in write mode since the stream is already open in write mode. When another instance tries to open it, it may either fail or succeed, we don't know. So the behavior is undefined.

So that's why we need to ensure that there is only one instance of the logger. Now, you may argue we can create a global instance of the logger and have the use it everywhere that may work. But it will not prevent the user from creating multiple instances of the logger. So the first thing we need to do is to prevent the user from creating instances of this class.

<font color="8C3061" >**Logger Implementation II -Preventing instantiation & copy**</font> 

In the previous lecture, we saw how multiple instances of the logger can corrupt the log file. So that's why we need to prevent the user from creating instance of the logger class.

**We have to do is just make the constructor private.** <br>
Now, after this, the user can no longer create the instances. So the *logger itself has to create the instance and provide that to the clients.*

*So one way is to create a static instance of the logger, this instance will be given to the clients through this static method.*

File:`Logger.h`

```c++
class Logger{
    FILE *m_pStream;
    std::string m_Tag;
    Logger() ;
public:
    ~Logger() ;
    void WriteLog(const char *pMessage) ;
    void SetTag(const char *pTag) ;
};
```

File:`main.h`

```c++
# include "Logger.h"

void OpenConnection()
{
    Logger &lg = Logger::Instance();
    lg.WriteLog("Attempting to open a connection");
}

int main()
{
    Logger &lg = Logger::Instance();
    lg.SetTag("192.168.1.101");
    lg.WriteLog("Application has started");
    OpenConnection();
    lg.WriteLog("Application is shutting down");
    return 0;
}
```

What if the user does not use a reference while acquiring the instance from the logger class?

```c++
Logger lg = Logger::Instance();
```

```c++
class Logger{
    FILE *m_pStream;
    std::string m_Tag;
    Logger();
>   static Logger m_Instance;
public:
>   Logger(const Logger &) = delete; // Prevent copying
>   Logger & operator=(const Logger &) = delete; // Prevent assignment
>   //Logger(Logger &&) = delete; // Prevent move construction
>   //Logger & operator=(Logger &&) = delete; // Prevent move assignment
>   static Logger & Instance();
    ~Logger() ;
    void WriteLog(const char *pMessage) ;
    void SetTag(const char *pTag) ;
};
```

Prevent copying, assignment, move construction and move assignment

<font color="8C3061" >**Logger Implementation III - Lazy Instantiation**</font> 


**The class itself is responsible for creating a singular instance.** <br>
Now in this particular implementation, this instance is created as soon as the program is loaded. This means the instance is created even before main gets invoked. That is why this type of instance is called as *eager instance*. 

File: `Logger.h`
```c++
static Logger *m_pInstance;    // Static instance pointer
```

File: `Logger.cpp`

```c++
Logger *Logger::m_pInstance;    // Static instance pointer
...
Logger & Logger::Instance(){ 
    if (m_pInstance == nullptr)     // Lazy initialization
        m_pInstance = new Logger(); // Create the instance
    return *m_pInstance;       // Return the instance
}

```

<font color="8C3061" >**Logger Implementation - Singleton Destruction Policies**</font> 


In the previous lecture, we realize that the destructor of logger is not invoked. 

So how do we ensure the instance is destroyed? And in this case, we want the instance to be destroyed because it has to invoke the destructor and we close the stream there.

 It is not a memory leak but assume your destructor is releasing some important resource. 
 
 Maybe it is setting an event or it is releasing a Semaphore and other applications are waiting on these objects. In this particular case, if the destructor is not invoked, **these resources will not be released and the other applications will get blocked.** 
 
 So that's why we may have to ensure that the destructor is invoked somehow and it can be invoked only when you delete the logger instance. Now there are two ways through which you can delete the logger instance.

<font color="8C3061" >**Smart Pointer (Difficult Method)**</font> 




File: `Logger.h`
```c++
//static Logger *m_pInstance;
inline static std::unique_prt<Logger> m_pInstance{} ;
```

This is in line static feature and was introduced in C 17. It allows you to initialize the static members in the class itself. You can also use the inline static with this declaration.

File: `Logger.h`
```c++
//static Logger *m_pInstance;
inline static std::unique_prt<Logger> m_pInstance{} ;
~Logger() ; // Destructor to close the log file to be private
```

File: `Logger.h`
```c++
//Logger *Logger::m_pInstance; 

// Instance returns the singleton instance of Logger
Logger & Logger::Instance(){ 
    if (m_pInstance == nullptr)     // Lazy initialization
>       //m_pInstance = new Logger();
        m_pInstance.reset(new Logger{}); // Use smart pointer to manage the instance
    return *m_pInstance;       // Return the instance
}
```

<font color="8C3061" >**Atexit and Lambda Function**</font> 

```c++
atexit([]() { 
        delete m_pInstance; // Ensure the instance is deleted at program exit
    });
```

# <font color="8C3061" >**Multithreading Issues**</font> 

The problem is two threads have called instance method at the same time. And if the instance is null `if(m_pInstance == nullptr)`, both will check instances null and reach this line `m_pInstance = new Logger{};`. 
And now there are two threads that will execute this statement. Consequently, two instances will be created.

File: `main.cpp`
```c++
std::thread t1{[](){
    Logger &lg = Logger::Instance();
    lg.WriteLog("Thread 1 has started");
}};
std::thread t2{[](){
    Logger &lg = Logger::Instance();
    lg.WriteLog("Thread 2 has started");
}};

t1.join();
t2.join();
```

<font color="8C3061" >**Use lock in this method**</font> 

Create a instance of `<mutex>` to thread safe.

File: `Logger.h`
```c++
class Logger{
    static std::mutex m_Mtx;
    ...
}
```

File: `Logger.cpp`
```c++
Logger *Logger::m_pInstance;  // Static instance pointer
std::mutex Logger::m_Mtx; // Mutex for thread safety

Logger & Logger::Instance(){
    m_Mtx.lock(); // Lock the mutex to ensure thread safety
    if (m_pInstance == nullptr)     // Lazy initialization
        m_pInstance = new Logger(); // Create the instance
    m_Mtx.unlock(); // Unlock the mutex
    return *m_pInstance;       // Return the instance
}

```

<div class="alert alert-block alert-info">
<b>Note:</b> Now one issue with this implementation is that after the instance is created when a thread calls the instance method, the thread will still acquire the lock on the mutex. And this is completely unnecessary after the instance is created because we no longer need to apply a lock because instance is not null and it will not go inside this condition. 

Therefore, if multiple threads call instance repeatedly, then it might have a negative impact on the application's performance  </div>

File: `Logger.cpp`
```c++

Logger & Logger::Instance(){
    //Double-checked locking pattern
    if (m_pInstance == nullptr){
        m_Mtx.lock(); // Lock the mutex to ensure thread safety
        if (m_pInstance == nullptr)     // Lazy initialization
            m_pInstance = new Logger(); // Create the instance
        m_Mtx.unlock(); // Unlock the mutex
    }
    return *m_pInstance;       // Return the instance
}

```